<a href="https://colab.research.google.com/github/Masoud-Ghasemian/PyTorch_Basics/blob/master/chatbot1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import random
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import nltk
import numpy as np
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
def tokenize(sentence):
  return nltk.word_tokenize(sentence)

In [3]:
sentence = 'hey how are you'
tokenize(sentence)

['hey', 'how', 'are', 'you']

In [4]:
def stem(word):
  return stemmer.stem(word.lower())

In [5]:
words = ['dogs', 'Cat', 'talked']
stemmed_words = [stem(word) for word in words]
print(stemmed_words)

['dog', 'cat', 'talk']


In [6]:
def bag_of_words(tokenized_sentence, all_words):
  bag = np.zeros(len(all_words), dtype=np.float32)
  tokenized_sentence = [stem(w) for w in tokenized_sentence]
  for idx, w in enumerate(all_words):
    if w in tokenized_sentence:
      bag[idx] = 1.0

  return bag

In [9]:
import json
with open('intents.json', 'r') as f:
  intents = json.load(f)

all_words = []
tags = []
xy = []
for intent in intents['intents']:
  tag = intent['tag']
  tags.append(tag)
  for pattern in intent['patterns']:
    w = tokenize(pattern)
    all_words.extend(w)
    xy.append((w, tag))
ignore_words = ['?', '!', ',', '.']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))
print(all_words)
print(tags)





FileNotFoundError: ignored

In [ ]:
X_train = []
y_train = []

for (pattern_sentence, tag) in xy:
  bag = bag_of_words(pattern_sentence, all_words)
  X_train.append(bag)

  label = tags.index(tag)
  y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)




In [ ]:
len(all_words)

In [ ]:
class ChatDataset(Dataset):
  def __init__(self):
    self.n_samples = len(X_train)
    self.x_data = X_train
    self.y_data = y_train

  def __getitem__(self, index):
    return self.x_data[index], self.y_data[index]

  def __len__(self):
    return self.n_samples


In [ ]:
batch_size = 8
hidden_size = 8
output_size = len(tags)
input_size = len(X_train[0])
learning_rate = 0.001



dataset = ChatDataset()
train_loader = DataLoader(dataset = dataset, batch_size=batch_size, shuffle=True, num_workers = 2)

In [8]:
class NeuralNet(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(NeuralNet, self).__init__()
    self.l1 = nn.Linear(input_size, hidden_size)
    self.l2 = nn.Linear(hidden_size, hidden_size)
    self.l3 = nn.Linear(hidden_size, num_classes)
    self.relu = nn.ReLU()

  def forward(self, x):
    out = self.relu(self.l1(x))
    out = self.relu(self.l2(out))
    return self.l3(out)
    

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.adam(model.parapeters(), lr = learning_rate)


for epoch in range(num_epochs):
  for (words, labels) in train_loader:
    words = words.to(device)
    labels = labels.to(device)

    #forwards
    outputs = model(words)
    loss = criterion(labels, outputs)

    #backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  if (epoch + 1) % 100 == 0:
    print(f'epoch {epoch + 1}/{num_epochs}, loss={loss.item():.4f}')

  
data = {
    "model_sate": model.state_dict(),
    "input_size": input_size,
    "output_size": output_size,
    "hidden_size": hidden_size,
    "all_words": all_words,
    "tags" : tags
}


FILE = 'data.pth'
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

In [ ]:
import random
